In [ ]:
import os
import argparse
import numpy as np
import torch
from torch import nn, optim

import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

import torchvision
from torchvision.utils import save_image
from torchvision import datasets, transforms
from torch.utils.data.dataloader import default_collate

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
class ModelD(nn.Module):
    def __init__(self):
        super(ModelD, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, 1, 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1  = nn.Linear(64*28*28+1000, 1024)
        self.fc2 = nn.Linear(1024, 1)
        self.fc3 = nn.Linear(10, 1000)

    def forward(self, x, labels):
        batch_size = x.size(0)
        x = x.view(batch_size, 1, 28,28)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = x.view(batch_size, 64*28*28)
        y_ = self.fc3(labels)
        y_ = F.relu(y_)
        x = torch.cat([x, y_], 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.sigmoid(x)

class ModelG(nn.Module):
    def __init__(self, z_dim):
        self.z_dim = z_dim
        super(ModelG, self).__init__()
        self.fc2 = nn.Linear(10, 1000)
        self.fc = nn.Linear(self.z_dim+1000, 64*28*28)
        self.bn1 = nn.BatchNorm2d(64)
        self.deconv1 = nn.ConvTranspose2d(64, 32, 5, 1, 2)
        self.bn2 = nn.BatchNorm2d(32)
        self.deconv2 = nn.ConvTranspose2d(32, 1, 5, 1, 2)

    def forward(self, x, labels):
        batch_size = x.size(0)
        y_ = self.fc2(labels)
        y_ = F.relu(y_)
        x = torch.cat([x, y_], 1)
        x = self.fc(x)
        x = x.view(batch_size, 64, 28, 28)
        x = self.bn1(x) 
        x = F.relu(x)
        x = self.deconv1(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.deconv2(x)
        x = F.sigmoid(x)
        return x

In [ ]:
batch_size = 128
lr = 0.01
epochs = 10
nz = 100
save_every = 1
save_dir = 'models'
samples_dir = 'samples'
cuda = True
print_every = 50

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

INPUT_SIZE = 784
SAMPLE_SIZE = 80
NUM_LABELS = 10

In [ ]:
def my_collate(batch):
    modified_batch = []
    for item in batch:
        image, label = item
        if label is not 9:
            modified_batch.append(item)
    return default_collate(modified_batch)

transform = transforms.Compose([
        transforms.ToTensor(),
])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform),
                   batch_size=128, shuffle=True, collate_fn = my_collate)

model_d = ModelD()
model_g = ModelG(nz)
criterion = nn.BCELoss()

input = torch.FloatTensor(batch_size, INPUT_SIZE)
noise = torch.FloatTensor(batch_size, (nz))

fixed_noise = torch.FloatTensor(SAMPLE_SIZE, nz).normal_(0,1)
fixed_labels = torch.zeros(SAMPLE_SIZE, NUM_LABELS)

for i in range(NUM_LABELS):
    for j in range(SAMPLE_SIZE // NUM_LABELS):
        fixed_labels[i*(SAMPLE_SIZE // NUM_LABELS) + j, i] = 1.0

label = torch.FloatTensor(batch_size)
one_hot_labels = torch.FloatTensor(batch_size, 10)

if cuda:
    model_d.cuda()
    model_g.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
    one_hot_labels = one_hot_labels.cuda()
    fixed_labels = fixed_labels.cuda()

optim_d = optim.SGD(model_d.parameters(), lr=lr)
optim_g = optim.SGD(model_g.parameters(), lr=lr)
fixed_noise = Variable(fixed_noise)
fixed_labels = Variable(fixed_labels)

real_label = 1
fake_label = 0

for epoch_idx in range(epochs):
    model_d.train()
    model_g.train()

    d_loss = 0.0
    g_loss = 0.0
    
    for batch_idx, (train_x, train_y) in enumerate(train_loader):
        batch_size = train_x.size(0)
        train_x = train_x.view(-1, INPUT_SIZE)
        if cuda:
            train_x = train_x.cuda()
            train_y = train_y.cuda()

        input.resize_as_(train_x).copy_(train_x)
        label.resize_(batch_size).fill_(real_label)
        one_hot_labels.resize_(batch_size, NUM_LABELS).zero_()
        one_hot_labels.scatter_(1, train_y.view(batch_size,1), 1)
        inputv = Variable(input)
        labelv = Variable(label)

        output = model_d(inputv, Variable(one_hot_labels))
        optim_d.zero_grad()
        errD_real = criterion(output, labelv)
        errD_real.backward()
        realD_mean = output.data.cpu().mean()

        one_hot_labels.zero_()
        rand_y = torch.from_numpy(
            np.random.randint(0, NUM_LABELS, size=(batch_size,1))).cuda()
        one_hot_labels.scatter_(1, rand_y.view(batch_size,1), 1)
        noise.resize_(batch_size, nz).normal_(0,1)
        label.resize_(batch_size).fill_(fake_label)
        noisev = Variable(noise)
        labelv = Variable(label)
        onehotv = Variable(one_hot_labels)
        g_out = model_g(noisev, onehotv)
        output = model_d(g_out, onehotv)
        errD_fake = criterion(output, labelv)
        fakeD_mean = output.data.cpu().mean()
        errD = errD_real + errD_fake
        errD_fake.backward()
        optim_d.step()

        # train the G
        noise.normal_(0,1)
        one_hot_labels.zero_()
        rand_y = torch.from_numpy(
            np.random.randint(0, NUM_LABELS, size=(batch_size,1))).cuda()
        one_hot_labels.scatter_(1, rand_y.view(batch_size,1), 1)
        label.resize_(batch_size).fill_(real_label)
        onehotv = Variable(one_hot_labels)
        noisev = Variable(noise)
        labelv = Variable(label)
        g_out = model_g(noisev, onehotv)
        output = model_d(g_out, onehotv)
        errG = criterion(output, labelv)
        optim_g.zero_grad()
        errG.backward()
        optim_g.step()

        d_loss += errD.data
        g_loss += errG.data
        if batch_idx % print_every == 0:
            print(
            "\t{} ({} / {}) mean D(fake) = {:.4f}, mean D(real) = {:.4f}".
                format(epoch_idx, batch_idx, len(train_loader), fakeD_mean,
                    realD_mean))

            g_out = model_g(fixed_noise, fixed_labels).data.view(
                SAMPLE_SIZE, 1, 28,28).cpu()
            save_image(g_out,
                '{}/{}_{}.png'.format(
                    samples_dir, epoch_idx, batch_idx))


    print('Epoch {} - D loss = {:.4f}, G loss = {:.4f}'.format(epoch_idx,
        d_loss, g_loss))
    if epoch_idx % save_every == 0:
        torch.save(model_d.state_dict(),
                    '{}/model_d_epoch_{}.pkl'.format(
                        save_dir, epoch_idx))
        torch.save(model_g.state_dict(),
                    '{}/model_g_epoch_{}.pkl'.format(
                        save_dir, epoch_idx))

## Transfer-Learning Part

In [ ]:
g_model = ModelG(100)
g_model.load_state_dict(torch.load("models/model_g_epoch_9.pkl"))
g_model.eval()

d_model = ModelD()
d_model.load_state_dict(torch.load("models/model_d_epoch_9.pkl"))
d_model.eval()

In [ ]:
g_model.cuda()
d_model.cuda()

In [ ]:
# initializing parameters
batch_size = 1

# Getting some images of nine ready for inference:

transform = transforms.Compose([
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST(root='nine_data', train=True, download=True, transform=transform)

idx = dataset.targets == 9

dataset.data = dataset.data[idx]
dataset.targets = dataset.targets[idx]

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
train_iter = iter(train_loader)
print(type(train_iter))

images, labels = train_iter.next()

print('images shape on batch size = {}'.format(images.size()))
print('labels shape on batch size = {}'.format(labels.size()))

In [ ]:
import torchvision
import matplotlib.pyplot as plt

grid = torchvision.utils.make_grid(images)

plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.axis('off')
plt.title(labels.numpy())

In [ ]:
# see if train_loader is giving only nines
print(type(train_loader))

count = 0

for x_, y in train_loader:
    count+= 1
    # print(x_)
    print(y)
    if count == 5:
        break

In [ ]:
# create data to be trained upon: around 200 images

train_custom_data = []

for i in range(1, 10):
    count = 0
    for item in train_loader:
        train_custom_data.append(item)
        count += 1
        if count == 4:
            break

In [ ]:
print(len(train_custom_data))

In [ ]:
input = torch.FloatTensor(batch_size, INPUT_SIZE)
noise = torch.FloatTensor(batch_size, (nz))

fixed_noise = torch.FloatTensor(SAMPLE_SIZE, nz).normal_(0,1)
fixed_labels = torch.zeros(SAMPLE_SIZE, NUM_LABELS)

for i in range(NUM_LABELS):
    for j in range(SAMPLE_SIZE // NUM_LABELS):
        fixed_labels[i*(SAMPLE_SIZE // NUM_LABELS) + j, i] = 1.0

label = torch.FloatTensor(batch_size)
one_hot_labels = torch.FloatTensor(batch_size, 10)

if cuda:
    model_d.cuda()
    model_g.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
    one_hot_labels = one_hot_labels.cuda()
    fixed_labels = fixed_labels.cuda()

optim_d = optim.SGD(model_d.parameters(), lr=lr)
optim_g = optim.SGD(model_g.parameters(), lr=lr)
fixed_noise = Variable(fixed_noise)
fixed_labels = Variable(fixed_labels)

real_label = 1
fake_label = 0

for epoch_idx in range(epochs):
    model_d.train()
    model_g.train()


    d_loss = 0.0
    g_loss = 0.0
    for batch_idx, (train_x, train_y) in enumerate(train_custom_data):
        batch_size = train_x.size(0)
        train_x = train_x.view(-1, INPUT_SIZE)
        if cuda:
            train_x = train_x.cuda()
            train_y = train_y.cuda()

        input.resize_as_(train_x).copy_(train_x)
        label.resize_(batch_size).fill_(real_label)
        one_hot_labels.resize_(batch_size, NUM_LABELS).zero_()
        one_hot_labels.scatter_(1, train_y.view(batch_size,1), 1)
        inputv = Variable(input)
        labelv = Variable(label)

        output = model_d(inputv, Variable(one_hot_labels))
        optim_d.zero_grad()
        errD_real = criterion(output, labelv)
        errD_real.backward()
        realD_mean = output.data.cpu().mean()

        one_hot_labels.zero_()
        rand_y = torch.from_numpy(
            np.random.randint(0, NUM_LABELS, size=(batch_size,1))).cuda()
        one_hot_labels.scatter_(1, rand_y.view(batch_size,1), 1)
        noise.resize_(batch_size, nz).normal_(0,1)
        label.resize_(batch_size).fill_(fake_label)
        noisev = Variable(noise)
        labelv = Variable(label)
        onehotv = Variable(one_hot_labels)
        g_out = model_g(noisev, onehotv)
        output = model_d(g_out, onehotv)
        errD_fake = criterion(output, labelv)
        fakeD_mean = output.data.cpu().mean()
        errD = errD_real + errD_fake
        errD_fake.backward()
        optim_d.step()

        # train the G
        noise.normal_(0,1)
        one_hot_labels.zero_()
        rand_y = torch.from_numpy(
            np.random.randint(0, NUM_LABELS, size=(batch_size,1))).cuda()
        one_hot_labels.scatter_(1, rand_y.view(batch_size,1), 1)
        label.resize_(batch_size).fill_(real_label)
        onehotv = Variable(one_hot_labels)
        noisev = Variable(noise)
        labelv = Variable(label)
        g_out = model_g(noisev, onehotv)
        output = model_d(g_out, onehotv)
        errG = criterion(output, labelv)
        optim_g.zero_grad()
        errG.backward()
        optim_g.step()

        d_loss += errD.data
        g_loss += errG.data
        if batch_idx % print_every == 0:
            print(
            "\t{} ({} / {}) mean D(fake) = {:.4f}, mean D(real) = {:.4f}".
                format(epoch_idx, batch_idx, len(train_loader), fakeD_mean,
                    realD_mean))

            g_out = model_g(fixed_noise, fixed_labels).data.view(
                SAMPLE_SIZE, 1, 28,28).cpu()
            save_image(g_out,
                '{}/{}_{}+nine.png'.format(
                    samples_dir, epoch_idx, batch_idx))


    print('Epoch {} - D loss = {:.4f}, G loss = {:.4f}'.format(epoch_idx,
        d_loss, g_loss))

In [ ]:
import imageio

images = []
for e in range(epochs-1):
    img_name = 'samples/' + str(e + 1) + '_0+nine.png' 
    images.append(imageio.imread(img_name))
imageio.mimsave('hahaha_generation_animation.gif', images, fps=1)